# Web Crawling
1. Beautiful Soup
2. **Selenium**
3. using API

# 2번째 방법 -  Selenium

Selenium 공식 홈페이지 : https://www.seleniumhq.org/  
Google Chrome Driver : https://chromedriver.storage.googleapis.com/index.html?path=2.45/

## Selenium 설치하기
```
pip install selenium 
conda install selenium # 아나콘다 사용자일 경우
```

## 왜 Selenium인가?
```
Selenium은 주로 웹앱을 테스트하는데 이용하는 프레임워크다. webdriver라는 API를 통해 운영체제에 설치된 Chrome등의 브라우저를 제어하게 된다.

브라우저를 직접 동작시킨다는 것은 JavaScript를 이용해 비동기적으로 혹은 뒤늦게 불러와지는 컨텐츠들을 가져올 수 있다는 것이다. 즉, ‘눈에 보이는’ 컨텐츠라면 모두 가져올 수 있다는 뜻이다. 우리가 requests에서 사용했던 .text의 경우 브라우저에서 ‘소스보기’를 한 것과 같이 동작하여, JS등을 통해 동적으로 DOM이 변화한 이후의 HTML을 보여주지 않는다. 반면 Selenium은 실제 웹 브라우저가 동작하기 때문에 JS로 렌더링이 완료된 후의 DOM결과물에 접근이 가능하다.

참고 : https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/
```
즉, Selenium은 instagram의 좋아요를 자동으로 눌러주는 것과 같이 **단순히 브라우저 내의 HTML & CSS만 긁어서는 할 수 없는 일들**을 한다.
하지만, 이번 예제는 Beautiful Soup을 사용해도 될만큼 간단한 예제를 준비해봤다.

Youtube에서 **BOAZ 관련된 영상의 좋아요 누르기를 자동화**해보자.

## Crawling하기 위한 준비

In [58]:
import numpy as np # pandas 라이브러리 의존성 때문에
import pandas as pd # 가져온 데이터를 가공하기 위해
from tqdm import tqdm_notebook # 진행률을 표시하기 위해
from selenium import webdriver as wd # Chrome 웹드라이버 준비
import time # selenium은 sleep을 걸어야해서
import urllib

In [75]:
# 접속할 사이트
base_url = 'https://www.youtube.com/results'
params = '?search_query=%s'
keyword = urllib.parse.quote('BOAZ 빅데이터')
target_url = base_url + (params%keyword)

# Selenium 드라이버
driver = wd.Chrome('./tool/chromedriver')

# 접속
driver.get(target_url)

---

## YouTube에서 영상 제목 수집하기

### 브라우저를 움직이게 해주는 함수
- execute_script 함수

In [76]:
# 스크롤 이벤트
for n in range(5):
    driver.execute_script('window.scrollBy(0,1000)') # 자바스크립트를 돌리는 코드
    time.sleep(1) # selenium에서는 sleep을 걸어주지 않으면 bot으로 인식당할 수 있음

### 정보를 찾아주는 함수
- find_elements_by_id
- find_elements_by_tag_name
- find_element_by_css_selector

selenium에서는 위와 같은 함수를 사용한다.  
이 함수를 통해서 Youtube 내 BOAZ 관련된 영상의 제목들을 찾아오자

In [77]:
# Youtube에 검색된 BOAZ 관련 영상들
videos = driver.find_elements_by_tag_name('ytd-video-renderer') # ytd-playlist-renderer안에는 좋아요가 없으므로
print('%d개의 영상을 찾았습니다.'%len(videos))

31개의 영상을 찾았습니다.


In [78]:
boaz_videos = list()
boaz_playlist = list()
for v in videos:
    # BOAZ와 무관한 영상들에 관해서 처리
    # 영상제목에 BOAZ나 boaz가 있을시 해당
    if (('BOAZ' in video_title) or ('boaz' in video_title) or ('보아즈' in video_title)):
        boaz_videos.append(v)

In [79]:
for v in boaz_videos:
    boaz_video_title = v.find_element_by_id('video-title').text
    boaz_video_url = v.find_element_by_id('video-title').get_attribute('href')
    boaz_playlist.append([boaz_video_title,boaz_video_url])

In [80]:
df = pd.DataFrame(boaz_playlist)
df.columns = ('Title','URL')
df

,Title,URL
0,제9회 보아즈(BOAZ) 빅데이터 컨퍼런스 - 너의 기분 이모지(emoji)?,https://www.youtube.com/watch?v=PElfNl7bH-w
1,제9회 보아즈(BOAZ) 빅데이터 컨퍼런스 - 강화학습을 이용한 주식 트레이딩 알고리즘,https://www.youtube.com/watch?v=5x6nbN-6tFU
2,제2회 BOAZ 빅데이터 컨퍼런스: 4. 프리미어리그 축구팀 승자예측,https://www.youtube.com/watch?v=sCy9-EFco2s
3,제 8회 BOAZ 빅데이터 컨퍼런스 -05 브랜드 불만지수 구축,https://www.youtube.com/watch?v=MvtFUeJyxd8
4,제 9회 보아즈(BOAZ) 빅데이터 컨퍼런스 - AiRBnB : Aspect in ...,https://www.youtube.com/watch?v=jhmbXiAYUq4
5,제9회 보아즈(BOAZ) 빅데이터 컨퍼런스 - 트위터 트렌드 분석을 통한 신제품 개발,https://www.youtube.com/watch?v=_sMPLpl5tLw
6,제 9회 보아즈(BOAZ) 빅데이터 컨퍼런스 - Deep Brother 인공지능 출석체크,https://www.youtube.com/watch?v=Iv9j24qrWRE
7,제 8회 BOAZ 빅데이터 컨퍼런스 -03 리뷰왕 딥리뷰,https://www.youtube.com/watch?v=aZnkJ111LS0
8,제 6회 BOAZ 빅데이터 컨퍼런스 - 03 홈런치기 좋은 날 - 날씨 변화와 야구...,https://www.youtube.com/watch?v=Kkvo0fFtSQI
9,제 6회 BOAZ 빅데이터 컨퍼런스 - 05 DeepTitle : 한국어 기사 자동 요약,https://www.youtube.com/watch?v=4XTVVjru6rc


--- 

## YouTube 로그인하기

사실 beautiful soup을 통해서도 로그인을 할 수 있다.
1. 다만 POST나 GET과 같은 개념을 이해하기엔 주어진 시간이 너무 짧기 때문에 생략했고
2. selenium으로 하는게 더욱 시각적으로 쉽기 때문이다

bs4를 통해서 로그인하는 방법을 알고 싶다면 다음 링크를 참고하자
https://beomi.github.io/2017/01/20/HowToMakeWebCrawler-With-Login/

### 움직일 수 있게 해주는 함수들
- click 함수
- send_keys 함수
- forward 함수
- back 함수

이제 조금더 selenium을 통해서 동적으로 활용해보자.

Ref : https://selenium-python.readthedocs.io/navigating.html#navigation-history-and-location

In [74]:
videos = driver.find_elements_by_id('video-title')
for v in videos:
    v.click()
    buttons = driver.find_elements_by_class('style-scope yt-icon-button')
    for b in buttons:
        print(b)

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_class'